In [1]:
import tkinter as tk
from tkinter import ttk, simpledialog
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sympy import symbols, Eq, solve
from tkinter import messagebox

class CalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Calculator")
        self.root.geometry("800x1000")
        self.create_widgets()

    def create_widgets(self):
        # Create a Notebook (tabbed interface)
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill="both", expand=True)

        # Create tabs for different modes
        self.create_standard_tab()  # Standard calculator
        self.create_scientific_tab()  # Scientific calculator
        self.create_matrix_tab()  # Matrix operations tab
        self.create_vector_tab()  # Vector operations tab
        self.create_graph_tab()  # Graphing tab
        self.create_equation_solving_tab()  # Equation solving tab

    def create_standard_tab(self):
        standard_tab = ttk.Frame(self.notebook)
        self.notebook.add(standard_tab, text="Standard")

        # Display area for standard mode
        self.display = tk.Entry(standard_tab, width=40, font=("Arial", 20), borderwidth=2, relief="solid", justify="right")
        self.display.grid(row=0, column=0, columnspan=4)

        # Buttons for Standard mode (Basic arithmetic operations)
        buttons = [
            ("7", 1, 0), ("8", 1, 1), ("9", 1, 2), ("/", 1, 3),
            ("4", 2, 0), ("5", 2, 1), ("6", 2, 2), ("*", 2, 3),
            ("1", 3, 0), ("2", 3, 1), ("3", 3, 2), ("-", 3, 3),
            ("0", 4, 0), (".", 4, 1), ("=", 4, 2), ("+", 4, 3),
            ("C", 5, 0)
        ]
        for (text, row, col) in buttons:
            button = tk.Button(standard_tab, text=text, width=10, height=2, font=("Arial", 14), command=lambda t=text: self.on_standard_button_click(t))
            button.grid(row=row, column=col)

    def create_scientific_tab(self):
        scientific_tab = ttk.Frame(self.notebook)
        self.notebook.add(scientific_tab, text="Scientific")

        # Display area for scientific mode
        self.display_scientific = tk.Entry(scientific_tab, width=40, font=("Arial", 20), borderwidth=2, relief="solid", justify="right")
        self.display_scientific.grid(row=0, column=0, columnspan=4)

        # Buttons for Scientific mode (Trigonometric, Logarithmic, etc.)
        buttons = [
            ("sin", 1, 0), ("cos", 1, 1), ("tan", 1, 2), ("log", 1, 3),
            ("sqrt", 2, 0), ("exp", 2, 1), ("π", 2, 2), ("^", 2, 3),
            ("7", 3, 0), ("8", 3, 1), ("9", 3, 2), ("/", 3, 3),
            ("4", 4, 0), ("5", 4, 1), ("6", 4, 2), ("*", 4, 3),
            ("1", 5, 0), ("2", 5, 1), ("3", 5, 2), ("-", 5, 3),
            ("0", 6, 0), (".", 6, 1), ("=", 6, 2), ("+", 6, 3),
            ("C", 7, 0)
        ]
        for (text, row, col) in buttons:
            button = tk.Button(scientific_tab, text=text, width=10, height=2, font=("Arial", 14), command=lambda t=text: self.on_scientific_button_click(t))
            button.grid(row=row, column=col)

    def create_matrix_tab(self):
        matrix_tab = ttk.Frame(self.notebook)
        self.notebook.add(matrix_tab, text="Matrix Operations")

        # Title label
        title_label = tk.Label(matrix_tab, text="Enter Matrix (Row by Row)", font=("Arial", 16))
        title_label.grid(row=0, column=0, columnspan=4, pady=10)

        # Matrix input fields (3x3 matrix)
        self.matrix_entries = []
        for i in range(3):
            row_entries = []
            for j in range(3):
                entry = tk.Entry(matrix_tab, width=10, font=("Arial", 14), borderwidth=2, relief="solid")
                entry.grid(row=i+1, column=j, padx=5, pady=5)
                row_entries.append(entry)
            self.matrix_entries.append(row_entries)

        # Buttons for Matrix operations
        self.result_label = tk.Label(matrix_tab, text="Result: ", font=("Arial", 14))
        self.result_label.grid(row=4, column=0, columnspan=4, pady=10)

        buttons = [
            ("Determinant", self.calculate_determinant),
            ("Inverse", self.calculate_inverse),
            ("Transpose", self.calculate_transpose),
            ("Clear", self.clear_matrix)
        ]

        for i, (text, command) in enumerate(buttons):
            button = tk.Button(matrix_tab, text=text, width=15, height=2, font=("Arial", 14), command=command)
            button.grid(row=5, column=i, padx=10, pady=10)

    def create_vector_tab(self):
        vector_tab = ttk.Frame(self.notebook)
        self.notebook.add(vector_tab, text="Vector Operations")

        title_label = tk.Label(vector_tab, text="Enter Vector Components", font=("Arial", 16))
        title_label.grid(row=0, column=0, columnspan=4, pady=10)

        # Vector input fields (3-component vector)
        self.vector_entries = []
        for i in range(3):
            entry = tk.Entry(vector_tab, width=10, font=("Arial", 14), borderwidth=2, relief="solid")
            entry.grid(row=i+1, column=0, padx=5, pady=5)
            self.vector_entries.append(entry)

        # Buttons for Vector operations
        self.vector_result_label = tk.Label(vector_tab, text="Result: ", font=("Arial", 14))
        self.vector_result_label.grid(row=5, column=0, columnspan=4, pady=10)

        buttons = [
            ("Dot Product", self.calculate_dot_product),
            ("Cross Product", self.calculate_cross_product),
            ("Add Vector", self.add_vector),
            ("Clear", self.clear_vector)
        ]

        for i, (text, command) in enumerate(buttons):
            button = tk.Button(vector_tab, text=text, width=15, height=2, font=("Arial", 14), command=command)
            button.grid(row=6, column=i, padx=10, pady=10)

    def create_graph_tab(self):
        graph_tab = ttk.Frame(self.notebook)
        self.notebook.add(graph_tab, text="Graphing")

        title_label = tk.Label(graph_tab, text="Enter Equation to Graph", font=("Arial", 16))
        title_label.grid(row=0, column=0, columnspan=4, pady=10)

        self.graph_display = tk.Entry(graph_tab, width=40, font=("Arial", 20), borderwidth=2, relief="solid", justify="right")
        self.graph_display.grid(row=1, column=0, columnspan=4, pady=10)

        # Button to plot the graph
        graph_button = tk.Button(graph_tab, text="Plot Graph", font=("Arial", 14), command=self.plot_graph)
        graph_button.grid(row=2, column=0, columnspan=4, pady=10)

        # Label to display error message if any
        self.graph_error_label = tk.Label(graph_tab, text="", font=("Arial", 14), fg="red")
        self.graph_error_label.grid(row=3, column=0, columnspan=4)

        # Canvas to display the graph
        self.canvas_frame = tk.Frame(graph_tab)
        self.canvas_frame.grid(row=4, column=0, columnspan=4, pady=10)

    def create_equation_solving_tab(self):
        equation_tab = ttk.Frame(self.notebook)
        self.notebook.add(equation_tab, text="Equation Solving")

        self.display_equation = tk.Entry(equation_tab, width=40, font=("Arial", 20), borderwidth=2, relief="solid", justify="right")
        self.display_equation.grid(row=0, column=0, columnspan=4)

        buttons = [
            ("Linear", 1, 0), ("Quadratic", 1, 1), ("Derivative", 1, 2), ("Integral", 1, 3),
            ("C", 2, 0, 4)
        ]
        for button in buttons:
            if len(button) == 4:
                text, row, col, colspan = button
                button_widget = tk.Button(equation_tab, text=text, width=10, height=2, font=("Arial", 14), command=lambda t=text: self.on_equation_button_click(t))
                button_widget.grid(row=row, column=col, columnspan=colspan)
            else:
                text, row, col = button
                button_widget = tk.Button(equation_tab, text=text, width=10, height=2, font=("Arial", 14), command=lambda t=text: self.on_equation_button_click(t))
                button_widget.grid(row=row, column=col)

    def on_standard_button_click(self, text):
        current_text = self.display.get()
        if text == "C":
            self.display.delete(0, tk.END)
        elif text == "=":
            try:
                result = eval(current_text)
                self.display.delete(0, tk.END)
                self.display.insert(tk.END, str(result))
            except Exception as e:
                self.display.delete(0, tk.END)
                self.display.insert(tk.END, "Error")
        else:
            self.display.insert(tk.END, text)

    def on_scientific_button_click(self, text):
        current_text = self.display_scientific.get()
        if text == "C":
            self.display_scientific.delete(0, tk.END)
        elif text == "=":
            try:
                result = eval(current_text)
                self.display_scientific.delete(0, tk.END)
                self.display_scientific.insert(tk.END, str(result))
            except Exception as e:
                self.display_scientific.delete(0, tk.END)
                self.display_scientific.insert(tk.END, "Error")
        else:
            self.display_scientific.insert(tk.END, text)

    def get_matrix_values(self):
        matrix = []
        for row in self.matrix_entries:
            values = []
            for entry in row:
                try:
                    values.append(float(entry.get()))
                except ValueError:
                    values.append(0)
            matrix.append(values)
        return np.array(matrix)

    def get_vector_values(self):
        vector = []
        for entry in self.vector_entries:
            try:
                vector.append(float(entry.get()))
            except ValueError:
                vector.append(0)
        return np.array(vector)

    def calculate_determinant(self):
        matrix = self.get_matrix_values()
        det = np.linalg.det(matrix)
        self.result_label.config(text=f"Determinant: {det:.2f}")

    def calculate_inverse(self):
        matrix = self.get_matrix_values()
        try:
            inv = np.linalg.inv(matrix)
            self.result_label.config(text=f"Inverse:\n{inv}")
        except np.linalg.LinAlgError:
            self.result_label.config(text="Inverse: Not possible (singular matrix)")

    def calculate_transpose(self):
        matrix = self.get_matrix_values()
        transpose = np.transpose(matrix)
        self.result_label.config(text=f"Transpose:\n{transpose}")

    def calculate_dot_product(self):
        vector1 = self.get_vector_values()
        vector2 = self.get_vector_values()
        dot_product = np.dot(vector1, vector2)
        self.vector_result_label.config(text=f"Dot Product: {dot_product:.2f}")

    def calculate_cross_product(self):
        vector1 = self.get_vector_values()
        vector2 = self.get_vector_values()
        cross_product = np.cross(vector1, vector2)
        self.vector_result_label.config(text=f"Cross Product: {cross_product}")

    def add_vector(self):
        vector1 = self.get_vector_values()
        vector2 = self.get_vector_values()
        sum_vector = vector1 + vector2
        self.vector_result_label.config(text=f"Vector Addition Result: {sum_vector}")

    def clear_matrix(self):
        for row in self.matrix_entries:
            for entry in row:
                entry.delete(0, tk.END)
        self.result_label.config(text="Result: ")

    def clear_vector(self):
        for entry in self.vector_entries:
            entry.delete(0, tk.END)
        self.vector_result_label.config(text="Result: ")

    def plot_graph(self):
        try:
            equation = self.graph_display.get()
            x = np.linspace(-10, 10, 400)
            y = eval(equation.replace("x", "(x)"))

            # Clear previous graph
            for widget in self.canvas_frame.winfo_children():
                widget.destroy()

            # Create new figure and axis for the plot
            fig, ax = plt.subplots(figsize=(6, 4))
            ax.plot(x, y)
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            ax.set_title('Graph of ' + equation)
            ax.grid(True)

            # Embed the plot in the Tkinter window
            canvas = FigureCanvasTkAgg(fig, master=self.canvas_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

            self.graph_error_label.config(text="")
        except Exception as e:
            self.graph_error_label.config(text="Error in equation")

# Main Application
if __name__ == "__main__":
    root = tk.Tk()
    app = CalculatorApp(root)
    root.mainloop()
